In [2]:
import random

def fraseTA(nombre):
    f = random.choice([
        "Entonceeees....",
        "A ver, eso es del prework",
        "tu proyecto esta muy guay, pero....",
        "PARFAVAAR",
        "este lab es suuuuperguay, pero en mi bootcamp no lo hicimos",
        "venga va chicos",
        "¿pero no te gusta?, ¡si tiene colorines!"
    ])
    
    return f"{nombre}: {f}"


fraseTA("alumno")

TypeError: fraseTA() missing 1 required positional argument: 'nombre'

In [3]:
def conAlumnoRandom():
    a = random.choice(["Pepe","Juan","Ana","Marta"])
    return fraseTA(a)
    

In [4]:
conAlumnoRandom()

'Marta: Entonceeees....'

# Callback pattern

In [5]:
def conAlumnoRandom(fn):
    a = random.choice(["Pepe","Juan","Ana","Marta"])
    return fn(a)

conAlumnoRandom(fraseTA)

'Pepe: tu proyecto esta muy guay, pero....'

# Decorator Pattern

In [16]:
def conGenteRandom(fn):
    def wrapper(*nombres):
        for nombre in nombres:
            print(fn(nombre))
    return wrapper

q = conGenteRandom(fraseTA)
q("Luis","Carlos","Marta","Laura","Pepe")

Luis: venga va chicos
Carlos: ¿pero no te gusta?, ¡si tiene colorines!
Marta: A ver, eso es del prework
Laura: tu proyecto esta muy guay, pero....
Pepe: ¿pero no te gusta?, ¡si tiene colorines!


In [19]:
x = conGenteRandom(lambda s:s+"3")
x("Hola","Adios")

Hola3
Adios3


In [20]:
def saluda(nombre):
    return f"Hola {nombre}!"


conGenteRandom(saluda)("Laura","Marta")

Hola Laura!
Hola Marta!


In [21]:
@conGenteRandom
def saluda(nombre):
    return f"Hola {nombre}!"

saluda("Laura","Marta")

Hola Laura!
Hola Marta!


In [22]:
def noOp(fn):
    def wrapper():
        return None
    return wrapper

@noOp
def saluda(nombre):
    return f"Hola {nombre}!"


saluda()

In [29]:
def alwaysReturnFive(fn):
    def wrapper(q):
        fn(q)
        return 5
    return wrapper

@alwaysReturnFive
def saluda(nombre):
    print("EJECUTADA")
    return f"Hola {nombre}!"

saluda("Hola")

EJECUTADA


5

In [39]:
def debug(fn):
    def wrapper(*args,**kwargs):
        print("Args",args)
        print("Kwargs",kwargs)
        ret_val = fn(*args,**kwargs)
        print("return ->", ret_val)
        return ret_val
    return wrapper

@debug
def saluda(nombre, ta, lang="es"):
    if lang == "es":
        return f"Hola {nombre} de parte de {ta}!"
    else:
        return f"Hello {nombre} from {ta}!"

saluda("Pepe","Amanda",lang="en")

Args ('Pepe', 'Amanda')
Kwargs {'lang': 'en'}
return -> Hello Pepe from Amanda!


'Hello Pepe from Amanda!'

In [40]:
import pandas as pd
import numpy as np


df = pd.DataFrame(np.random.random((10,4)))
df

,0,1,2,3
0,0.114508,0.425395,0.998408,0.492482
1,0.241124,0.802920,0.636476,0.068403
2,0.454861,0.313134,0.134621,0.038291
3,0.615027,0.909404,0.459314,0.512180
4,0.127756,0.532356,0.614453,0.227362
5,0.966280,0.077251,0.653412,0.540416
6,0.058890,0.183286,0.682105,0.188074
7,0.670760,0.654455,0.489345,0.727280
8,0.633783,0.418822,0.480136,0.606129
9,0.660996,0.755232,0.407400,0.807000


In [57]:
@debug
def multiplicaPor10(value):
    print("call")
    return (value*10).astype("int")

df[[2]].apply(multiplicaPor10)


Args (0    0.998408
1    0.636476
2    0.134621
3    0.459314
4    0.614453
5    0.653412
6    0.682105
7    0.489345
8    0.480136
9    0.407400
Name: 2, dtype: float64,)
Kwargs {}
call
return -> 0    9
1    6
2    1
3    4
4    6
5    6
6    6
7    4
8    4
9    4
Name: 2, dtype: int64
Args (0    0.998408
1    0.636476
2    0.134621
3    0.459314
4    0.614453
5    0.653412
6    0.682105
7    0.489345
8    0.480136
9    0.407400
Name: 2, dtype: float64,)
Kwargs {}
call
return -> 0    9
1    6
2    1
3    4
4    6
5    6
6    6
7    4
8    4
9    4
Name: 2, dtype: int64


,2
0,9
1,6
2,1
3,4
4,6
5,6
6,6
7,4
8,4
9,4


In [68]:
class Vector:
    def __init__(self, *coord):
        self.coord = coord
        
    def check_len(fn):
        def wrapper(self,other):
            if not len(self.coord) == len(other.coord):
                raise ValueError("Vectors of different lengths")
            return fn(self,other)
        return wrapper
    
    def equals(self,other):
        return all([a==b for a,b in zip(self.coord,other.coord)])

    @debug
    @check_len
    def add(self,other):
        return Vector([a+b for a,b in zip(self.coord,other.coord)])
        
    @check_len
    def subtract(self,other):
        return Vector([a-b for a,b in zip(self.coord,other.coord)])

    @check_len
    def dot(self,other):
        return sum([a*b for a,b in zip(self.coord,other.coord)])
    
    def norm(self):
        return sum([a**2 for a in self.coord])**0.5
        
    def __str__(self):
        return f"({','.join(map(str,self.coord))})"

    def __repr__(self):
        return self.__str__()
    

a = Vector(1,2,3,9)
b = Vector(4,5,6,7)

a.add(b)

Args ((1,2,3,9), (4,5,6,7))
Kwargs {}
return -> ([5, 7, 9, 16])


([5, 7, 9, 16])

# Parametrized decorators

In [95]:
global debugEnabled
debugEnabled = False
def debug(fn):
    def wrapper(*args,**kwargs):
        if debugEnabled:
            print("=Debug of fucntion: ",fn.__name__)
            print("=Args",args)
            print("=Kwargs",kwargs)
        ret_val = fn(*args,**kwargs)
        if debugEnabled:
            print("=return ->", ret_val)
        return ret_val
    return wrapper

In [96]:
@debug
def saluda(nombre):
    return f"Hola {nombre}!"

for q in range(10):
    print(saluda(f"Hola {q}"))
    if q%2 == 0:
        debugEnabled = True
    else:
        debugEnabled = False


Hola Hola 0!
=Debug of fucntion:  saluda
=Args ('Hola 1',)
=Kwargs {}
=return -> Hola Hola 1!
Hola Hola 1!
Hola Hola 2!
=Debug of fucntion:  saluda
=Args ('Hola 3',)
=Kwargs {}
=return -> Hola Hola 3!
Hola Hola 3!
Hola Hola 4!
=Debug of fucntion:  saluda
=Args ('Hola 5',)
=Kwargs {}
=return -> Hola Hola 5!
Hola Hola 5!
Hola Hola 6!
=Debug of fucntion:  saluda
=Args ('Hola 7',)
=Kwargs {}
=return -> Hola Hola 7!
Hola Hola 7!
Hola Hola 8!
=Debug of fucntion:  saluda
=Args ('Hola 9',)
=Kwargs {}
=return -> Hola Hola 9!
Hola Hola 9!


In [98]:
a = lambda x: f"Hola {x}"

a("pepe")
print(a.__name__)

<lambda>


In [104]:

def checkPositionals(fn):
    def wrapper(*args,**kwargs):
        return fn(*args,**kwargs)
    return wrapper
    
    
@checkPositionals   
def suma(a,b):
    if type(a) != int or type(b) != int:
        raise ValueError("all positionals must be int")
    return a+b

suma(5,10)
suma("5",10)

ValueError: all positionals must be int

In [105]:

def checkPositionals(fn):
    def wrapper(*args,**kwargs):
        if type(args[0]) != int or type(args[1]) != int:
            raise ValueError("all positionals must be int")
        return fn(*args,**kwargs)
    return wrapper
    
    
@checkPositionals   
def suma(a,b):    
    return a+b

suma("5",10)

ValueError: all positionals must be int

In [120]:
def checkPositionals(fn):
    def wrapper(*args,**kwargs):
        types = [int, int]
        for posIndex,(t,arg) in enumerate(zip(types, args)):
            if type(arg) != t:
                a = args[posIndex]
                raise ValueError(f"Invalid type for '{a}' type is {type(arg)} and must be a {t}")
       
        return fn(*args,**kwargs)
    return wrapper
    
    
@checkPositionals   
def suma(a,b):    
    return a+b

suma(5,{})

ValueError: Invalid type for '{}' type is <class 'dict'> and must be a <class 'int'>

In [132]:
import random

def cantante(nombreCantante):
    frases = {
        "rosalia": ["tra tra","con altura","malamente"],
        "pitbul": ["Dale","1,2,3","Mr. Worlwide"]
    }
    def decorator(fn):
        def wrapper(nombre):
            frase = random.choice(frases[nombreCantante])
            return f"{frase} " + fn(nombre)
        return wrapper
    return decorator


@cantante("pitbul")
def saluda(nombre):
    return f"Hola {nombre}"

saluda("Pepe")

'Mr. Worlwide Hola Pepe'

In [136]:
def checkPositionals(*positionalTypes):
    def decorator(fn):
        def wrapper(*args,**kwargs):
            for posIndex,(t,arg) in enumerate(zip(positionalTypes, args)):
                if type(arg) != t:
                    a = args[posIndex]
                    raise ValueError(f"Invalid type for '{a}' type is {type(arg)} and must be a {t}")
            return fn(*args,**kwargs)
        return wrapper
    return decorator
    
    
@checkPositionals(int, int)
def suma(a,b):    
    return a+b

suma("5",5)

ValueError: Invalid type for '5' type is <class 'str'> and must be a <class 'int'>

In [148]:

df = pd.DataFrame(np.random.random((10,4)))


@checkPositionals(pd.Series)
def transformSerie(serie):
    return serie*10


display(df)
df2 = df[0].apply(transformSerie)
display(df2)


,0,1,2,3
0,0.373245,0.105874,0.178840,0.163848
1,0.156550,0.660046,0.184312,0.223642
2,0.546592,0.540364,0.815101,0.498135
3,0.296478,0.109945,0.631003,0.024810
4,0.940210,0.401925,0.420128,0.583510
5,0.593355,0.744431,0.511534,0.950598
6,0.944970,0.327291,0.140320,0.038348
7,0.027665,0.674195,0.074534,0.663475
8,0.517190,0.866111,0.624002,0.145348
9,0.185456,0.113676,0.015247,0.431757


ValueError: Invalid type for '0.3732447104901099' type is <class 'float'> and must be a <class 'pandas.core.series.Series'>

In [150]:
type(df[0])

pandas.core.series.Series

In [154]:
print(type(df[[0]]))
df[[0]]

<class 'pandas.core.frame.DataFrame'>


,0
0,0.373245
1,0.156550
2,0.546592
3,0.296478
4,0.940210
5,0.593355
6,0.944970
7,0.027665
8,0.517190
9,0.185456


# Sentry

In [1]:
import os
import dotenv
dotenv.load_dotenv()
import sentry_sdk
from sentry_sdk import capture_exception


SENTRY_DSN = os.getenv("SENTRY_DSN")
print(SENTRY_DSN)
sentry_sdk.init(SENTRY_DSN)

https://838bd3883316466eb0006e9e33510248@o153873.ingest.sentry.io/5208001


In [2]:
try:
    a = 8/0
except Exception as e:
    print(e)
    capture_exception(e)

division by zero


In [5]:
def captureError(fn):
    def wrapper(*args,**kwargs):
        try:
            return fn(*args,**kwargs)
        except Exception as e:
            capture_exception(e)
            raise e
    return wrapper


@captureError
def generateError():
    return list(range(10))[1000]
   
generateError()

IndexError: list index out of range